<a href="https://colab.research.google.com/github/google-deepmind/disentangled_rnns/blob/main/disentangled_rnns/notebooks/train_single_gru.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install disentangled_rnns repo from github
!pip install disentangled_rnns

import optax
import haiku as hk

from disentangled_rnns.library import rnn_utils
from disentangled_rnns.library import get_datasets
from disentangled_rnns.library import two_armed_bandits


# Define a dataset

In [ ]:
dataset = get_datasets.get_q_learning_dataset(n_sessions=500, n_trials=200)
dataset_train, dataset_eval = rnn_utils.split_dataset(dataset, eval_every_n=2)

# Define and train RNN

In [ ]:
# Define the architecture of the network we'd like to train
n_hidden = 16  # @param {type: "integer"}
output_size = 2

def make_network():
  model = hk.DeepRNN(
      [hk.GRU(n_hidden), hk.Linear(output_size=output_size)]
  )
  return model

In [ ]:
# INITIALIZE THE NETWORK
# Running rnn_utils.train_network with n_steps=0 does no training but sets up the
# parameters and optimizer state.
optimizer = optax.adam(learning_rate=1e-3)

params, opt_state, losses = rnn_utils.train_network(
    make_network = make_network,
    training_dataset=dataset_train,
    validation_dataset=dataset_eval,
    opt = optimizer,
    loss="categorical",
    n_steps=0)


In [ ]:
# TRAIN THE NETWORK
# Running this cell repeatedly continues to train the same network.
# The cell below gives insight into what's going on in your network.
# If you'd like to reinitialize the network and start over, re-run the above cell

n_steps = 1_000  # @param {type: "integer"}
learning_rate = 1e-3  # @param {type: "number"}

params, opt_state, losses = rnn_utils.train_network(
    make_network = make_network,
    training_dataset=dataset_train,
    validation_dataset=dataset_eval,
    loss="categorical",
    params=params,
    opt_state=opt_state,
    opt = optax.adam(learning_rate=learning_rate),
    loss_param = 1,
    n_steps=n_steps,
    do_plot = True)


In [ ]:
# Run forward pass on the unseen data
eval_data = dataset_eval.get_all()
xs_eval, ys_eval = eval_data['xs'], eval_data['ys']
network_output, network_states = rnn_utils.eval_network(make_network, params, xs_eval)

# Compute normalized likelihood
score = rnn_utils.normalized_likelihood(ys_eval, network_output)
print(f'Normalized Likelihood: {100*score:.1f}%')

# Plot network activations on an example session
example_session = 0  # @param {type: "integer"}

choices =  xs_eval[:, example_session, 0]
rewards = xs_eval[:, example_session, 1]
scalars = network_states[:, example_session, :]
two_armed_bandits.plot_2ab_sessdata(choices,
                                    rewards,
                                    scalars=scalars,
                                    scalar_types='agent_states',
                                    show_legend=False)